In [1]:
import sys
sys.path.append("../timm-efficientdet-pytorch")

import os

import pandas as pd

import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

from dataset import (
    XrayClassifyAdaptor,
    XrayDetectAdaptor,
    XrayInferenceAdaptor,
    XrayInferenceDataset
)
from datamodules import (
    XrayClassifyDataModule,
    XrayDetectDataModule,
)
from models import XrayClassifier, XrayDetector

import warnings
warnings.filterwarnings("ignore")

torch.set_float32_matmul_precision("medium")

In [2]:
DATA_DIR = "../data"
TRAIN_DF = "train_wbf.csv"
TRAIN_META = "train_meta.csv"
TEST_META = "test_meta.csv"

TRAIN_IMG_PATH = "train_512"
TEST_PATH = os.path.join(DATA_DIR, "test_512")

device = "cuda" if torch.cuda.is_available() else "cpu"

train_df = pd.read_csv(os.path.join(DATA_DIR, TRAIN_DF))
train_meta = pd.read_csv(os.path.join(DATA_DIR, TRAIN_META))
images_dir = os.path.join(DATA_DIR, TRAIN_IMG_PATH)
test_meta = pd.read_csv(os.path.join(DATA_DIR, TEST_META))

pl.seed_everything(0)

Global seed set to 0


0

# Classify

## Train

In [3]:
xca = XrayClassifyAdaptor(images_dir, train_df, train_meta)
xia = XrayInferenceAdaptor(TEST_PATH, test_meta)
xcm = XrayClassifyDataModule(xca)

classifier = XrayClassifier(model_name="tf_efficientnet_b0")

callbacks = [EarlyStopping(monitor="val_loss", patience=3, mode="min")]

trainer = pl.Trainer(accelerator="auto",
           devices="auto",
           precision="16-mixed",
           max_epochs=5,
           callbacks=callbacks)

trainer.fit(classifier, xcm)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/rg_im/Documents/coding/kaggle/VinBigData_Chest_Xray/code/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type         | Params
-----------------------------------------
0 | model   | EfficientNet | 4.0 M 
1 | metrics | ModuleDict   | 0     
-----------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.035    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


# Predict

In [4]:
pred_ds_clf = XrayInferenceDataset(xia)
pred_dl_clf = DataLoader(pred_ds_clf, batch_size=64, num_workers=8, shuffle=False)

trainer.predict(classifier, dataloaders=pred_dl_clf);
clf_result = classifier.predict_result.copy()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

---

# Detect

## Train

In [5]:
xda_tr = XrayDetectAdaptor(images_dir, train_df, train_meta, train=True)
xda_vl = XrayDetectAdaptor(images_dir, train_df, train_meta, train=False)
xdm = XrayDetectDataModule(xda_tr, xda_vl, batch_size=8)

detector = XrayDetector(architecture="tf_efficientnet_b5")

callbacks = [EarlyStopping(monitor="valid_loss", patience=10, mode="min")]

trainer = pl.Trainer(accelerator="auto",
                     precision="16-mixed",
                     devices="auto",
                     min_epochs=20,
                     max_epochs=100,
                     callbacks=callbacks)
trainer.fit(model=detector, datamodule=xdm)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type          | Params
----------------------------------------
0 | model | DetBenchTrain | 27.7 M
----------------------------------------
27.7 M    Trainable params
0         Non-trainable params
27.7 M    Total params
110.983   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

## Predict

In [ ]:
xia = XrayInferenceAdaptor(TEST_PATH, test_meta)
pred_ds_det = XrayInferenceDataset(xia)
pred_dl_det = DataLoader(pred_ds_det, batch_size=64, num_workers=8, shuffle=False)

trainer.predict(model=detector, dataloaders=pred_dl_det)
det_result = detector.predict_result.copy()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

# post-process

In [ ]:
# label, type, coordinates match
det_result["image_id"] = det_result["image_id"].map(lambda x: x.split(".")[0])
det_result["labels"] = det_result["labels"].map(lambda x: [int(y-1) for y in x] if sum(x) > 0 else [14])
det_result["confidences"] = det_result["confidences"].map(lambda x: [1] if sum(x)==0 else x)
det_result["box_coordinates"] = det_result["box_coordinates"] \
    .map(lambda x: [[0, 0, 1, 1]] if x == [[0, 0, 0, 0]] 
                                  else [list(map(int, y)) for y in x])

# convert to submission formet
results = {}
for _, row in det_result.iterrows():
    ans = ""
    temp = list(zip(row["labels"], row["confidences"], row["box_coordinates"]))
    for i in range(len(temp)):
        ans += " ".join(map(str, temp[i][:2])) + " " + " ".join(map(str, temp[i][2])) + " "
    ans = ans.rstrip()
    results[row["image_id"]] = ans

results = pd.DataFrame([results]).T \
    .reset_index() \
    .rename(columns={"index": "image_id", 0: "PredictionString"}) \
    .sort_values("image_id").reset_index(drop=True)

# only detector
results.to_csv("submission_one_stage.csv", index=False)

clf_result = pd.DataFrame([clf_result]).T \
    .reset_index() \
    .rename(columns={"index": "image_id", 0: "pred"}) \
    .sort_values("image_id").reset_index(drop=True)

# classifier + detector
results_ts = results.merge(clf_result, on="image_id")
results_ts.loc[results_ts["pred"] < 0.5, "PredictionString"] = "14 1 0 0 1 1"
results_ts = results_ts[["image_id", "PredictionString"]]
results_ts.to_csv("submission_two_stage.csv", index=False)

In [ ]:
results.head()

,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,0 0.82958984375 1258 579 1528 893 0 0.44970703...
2,008bdde2af2462e86fd373a445d0f4cd,0 0.7734375 1426 826 1755 1192 3 0.77001953125...
3,009bc039326338823ca3aa84381f17f1,0 0.7939453125 989 488 1215 740 3 0.6547851562...
4,00a2145de1886cb9eb88869c85d74080,0 0.82861328125 1109 710 1347 945 3 0.77734375...


In [ ]:
results_ts.head()

,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,14 1 0 0 1 1
2,008bdde2af2462e86fd373a445d0f4cd,0 0.7734375 1426 826 1755 1192 3 0.77001953125...
3,009bc039326338823ca3aa84381f17f1,14 1 0 0 1 1
4,00a2145de1886cb9eb88869c85d74080,0 0.82861328125 1109 710 1347 945 3 0.77734375...
